In [10]:
%pip install datasets
%pip install librosa

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 2.8/2.8 MB 32.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 11.1/11.1 MB 98.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/43.6 MB ? eta -:--:--
   ------------- -------------------------- 14.7/43.6 MB 71.0 MB/s eta 0:00:01
   -------------------- ------------------- 22.8/43.6 MB 55.4 MB/s eta 0:00:01
   ------------------------------- -------- 34.1/43.6 MB 54.1 MB/s eta 0:00:01
   ---------------------------------------  43.5/43.6 MB 55.3 MB/s eta 0:00:01
   ---------------------------------------- 43.6/43.6 MB 52.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 47.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/30.3 MB ? eta -:--:--
   -------------------- 


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from datasets import load_dataset
ds = load_dataset("diarizers-community/ami", "ihm")

print(ds)

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub\datasets--diarizers-community--ami. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an a

DatasetDict({
    train: Dataset({
        features: ['audio', 'timestamps_start', 'timestamps_end', 'speakers'],
        num_rows: 136
    })
    validation: Dataset({
        features: ['audio', 'timestamps_start', 'timestamps_end', 'speakers'],
        num_rows: 18
    })
    test: Dataset({
        features: ['audio', 'timestamps_start', 'timestamps_end', 'speakers'],
        num_rows: 16
    })
})


In [ ]:
ds['train'][0]

## Get wav file audio

In [41]:
import soundfile as sf

def export_as_wav_file(ds_entry, output_path = 'test.wav'):
    y = ds_entry['audio']['array']
    sr = ds_entry['audio']['sampling_rate']

    sf.write(output_path, y, sr)

ds_entry = ds['train'][0]
export_as_wav_file(ds_entry)

In [34]:
def find_unique_elements(lst):
    return list(set(lst))

unique_elements = find_unique_elements(ds['train'][0]['speakers'])

print(f"Number of speakers = {len(unique_elements)}")
unique_elements

Number of speakers = 4


['MIO026', 'MIE029', 'MIE083', 'MIE080']

## Get rttm file as ground truth

In [40]:
def form_rttm_file(df, file_id, output_path = 'test.rttm'):
    ''' Get rttm of 1 entry in ami'''

    timestamps_start = df['timestamps_start']
    timestamps_end = df['timestamps_end']
    speakers = df['speakers']

    for i in range(len(timestamps_start)):

        duration = timestamps_end[i] - timestamps_start[i]

        with open(output_path, 'a') as f:
            line = f"SPEAKER {file_id} 1 {timestamps_start[i]:.3f} {duration:.3f} <NA> <NA> {speakers[i]} <NA>\n"
            f.write(line)


form_rttm_file(ds['train'][0], ds['train'][0]['audio']['path'], 'test.rttm')

## Exporting out the dataset

In [ ]:
import soundfile as sf

def export_as_wav_file(ds_entry, output_path = 'test.wav'):
    ''' export 1 entry in ami'''
    y = ds_entry['audio']['array']
    sr = ds_entry['audio']['sampling_rate']

    sf.write(output_path, y, sr)

def form_rttm_file(df, file_id, output_path = 'test.rttm'):
    ''' Get rttm of 1 entry in ami'''

    timestamps_start = df['timestamps_start']
    timestamps_end = df['timestamps_end']
    speakers = df['speakers']

    for i in range(len(timestamps_start)):

        duration = timestamps_end[i] - timestamps_start[i]

        with open(output_path, 'a') as f:
            line = f"SPEAKER {file_id} 1 {timestamps_start[i]:.3f} {duration:.3f} <NA> <NA> {speakers[i]} <NA>\n"
            f.write(line)


